In [124]:
import numpy
import torch
import gym 
from stable_baselines3 import PPO
from stable_baselines3 import PPO 

In [125]:
env = gym.make("CartPole-v1")
model = PPO("MlpPolicy", env, verbose=1, device='cuda')
model.learn(total_timesteps=10000)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 20.5     |
|    ep_rew_mean     | 20.5     |
| time/              |          |
|    fps             | 448      |
|    iterations      | 1        |
|    time_elapsed    | 4        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 25.2        |
|    ep_rew_mean          | 25.2        |
| time/                   |             |
|    fps                  | 354         |
|    iterations           | 2           |
|    time_elapsed         | 11          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.009362972 |
|    clip_fraction        | 0.111       |
|    clip_range           | 0.2         |
|    entropy_loss  

In [126]:
custom_rewards = []
episodes = 10

for episode in range(episodes):
    state = env.reset()[0]
    done = False
    episode_reward = 0

    while not done:
        action, _ = model.predict(state, deterministic=True)
        state, reward, done, truncated, _ = env.step(action)
        episode_reward += reward

    custom_rewards.append(episode_reward)

avg_custom_reward = sum(custom_rewards) / episodes
print(f"Average reward : {avg_custom_reward}")

Average reward : 433.5


In [127]:
import numpy as np
from gym import RewardWrapper

class CustomCartPoleReward(RewardWrapper):
    def __init__(self, env):
        super(CustomCartPoleReward, self).__init__(env)

    def reward(self, reward):
        # Increase reward for keeping the pole upright and penalize for moving away from the center
        x, x_dot, theta, theta_dot = self.env.state
        new_reward = reward - np.abs(theta)  # Penalize for angle from upright
        return new_reward

# Initialize the custom environment
custom_env = CustomCartPoleReward(gym.make("CartPole-v1"))

In [128]:

custom_model = PPO("MlpPolicy", custom_env, verbose=1, device='cuda')
custom_model.learn(total_timesteps=10000)


Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 20.9     |
|    ep_rew_mean     | 19.2     |
| time/              |          |
|    fps             | 436      |
|    iterations      | 1        |
|    time_elapsed    | 4        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 25.1        |
|    ep_rew_mean          | 23.1        |
| time/                   |             |
|    fps                  | 350         |
|    iterations           | 2           |
|    time_elapsed         | 11          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008619834 |
|    clip_fraction        | 0.104       |
|    clip_range           | 0.2         |
|    entropy_loss  

In [129]:
custom_rewards = []
episodes = 10

for episode in range(episodes):
    state = custom_env.reset()[0]
    done = False
    episode_reward = 0

    while not done:
        action, _ = custom_model.predict(state, deterministic=True)
        state, reward, done, truncated, _ = custom_env.step(action)
        episode_reward += reward

    custom_rewards.append(episode_reward)

avg_custom_reward = sum(custom_rewards) / episodes
print(f"Average reward with custom reward function: {avg_custom_reward}")

Average reward with custom reward function: 555.6039355697585


In [130]:
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
import torch as th
from torch import nn
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.utils import set_random_seed

class CustomMLP(nn.Module):
    def __init__(self):
        super(CustomMLP, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(4, 64), nn.ReLU(),
            nn.Linear(64, 64), nn.ReLU(),
            nn.Linear(64, 64), nn.ReLU(),
            nn.Linear(64, 64), nn.ReLU(),
            nn.Linear(64, 2)
        ).to(th.device('cuda' if th.cuda.is_available() else 'cpu'))

    def forward(self, x):
        return self.network(x)

# Modify the policy architecture
model = PPO("MlpPolicy", env, policy_kwargs=dict(activation_fn=th.nn.ReLU, net_arch=[128, 128]), verbose=1, device='cuda')

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [131]:
model.learn(total_timesteps=10000)

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 21.4     |
|    ep_rew_mean     | 21.4     |
| time/              |          |
|    fps             | 389      |
|    iterations      | 1        |
|    time_elapsed    | 5        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 28.2        |
|    ep_rew_mean          | 28.2        |
| time/                   |             |
|    fps                  | 332         |
|    iterations           | 2           |
|    time_elapsed         | 12          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.010639362 |
|    clip_fraction        | 0.0746      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.687      |
|    explained_variance   | -0.0146     |
|    learning_rate        | 0.

In [132]:
custom_rewards = []
episodes = 10

for episode in range(episodes):
    state = env.reset()[0]
    done = False
    episode_reward = 0

    while not done:
        action, _ = model.predict(state, deterministic=True)
        state, reward, done, truncated, _ = env.step(action)
        episode_reward += reward

    custom_rewards.append(episode_reward)

avg_custom_reward = sum(custom_rewards) / episodes
print(f"Average reward : {avg_custom_reward}")

Average reward : 559.5
